In [28]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [29]:
import os
import http.client
import json

In [30]:
# ! pip install -U langchain-groq

In [31]:
# pick a model
from langchain.chat_models import init_chat_model
llm = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

In [32]:
from langchain_core.tools import tool

@tool
def browse(txt: str) -> str:
    """ Call this tool if you want to search the web for current information"""

    conn = http.client.HTTPSConnection("google.serper.dev")
    payload = json.dumps({
    "q": txt,
    "gl": "in"
    })
    headers = {
    'X-API-KEY': os.environ.get("SERPER_API_KEY"),
    'Content-Type': 'application/json'
    }
    conn.request("POST", "/search", payload, headers)
    res = conn.getresponse()
    data = res.read()
    #print(data.decode("utf-8"))
    data_dict = json.loads(data.decode("utf-8"))
    return data_dict['organic'][0]['snippet']


In [33]:
tools_list = [browse]
tools_dict = {t.name: t for t in tools_list} # comes in handy at the time of invokation

In [34]:
# we create a tool calling Agent by binding a list of tools to the llm
llm_with_tools = llm.bind_tools(tools_list)

In [35]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage

# This will store all converation
chat_history = [
    SystemMessage(content="You are a chatbot. Depending on the conversation, if user demands current information, you must use the tool 'browse' to get the information. If you do not need current information, you can answer directly. Always try to keep the conversation short and crisp.")
    ]

In [36]:
chat_history=[]
llm_with_tools = llm.bind_tools(tools_list)

In [37]:
chat_history.append(HumanMessage(content="Tell me about the effects of air pollution"))

response = llm_with_tools.invoke(chat_history)
response


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '4ywwmyaaz', 'function': {'arguments': '{"txt":"effects of air pollution"}', 'name': 'browse'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 232, 'total_tokens': 249, 'completion_time': 0.065984086, 'prompt_time': 0.013846616, 'queue_time': 0.19894189, 'total_time': 0.079830702}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_9e1e8f8435', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4ba9fc1a-4f3e-4173-84a4-0460290c8f6d-0', tool_calls=[{'name': 'browse', 'args': {'txt': 'effects of air pollution'}, 'id': '4ywwmyaaz', 'type': 'tool_call'}], usage_metadata={'input_tokens': 232, 'output_tokens': 17, 'total_tokens': 249})

In [38]:
chat_history.append(response)
chat_history


[HumanMessage(content='Tell me about the effects of air pollution', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '4ywwmyaaz', 'function': {'arguments': '{"txt":"effects of air pollution"}', 'name': 'browse'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 232, 'total_tokens': 249, 'completion_time': 0.065984086, 'prompt_time': 0.013846616, 'queue_time': 0.19894189, 'total_time': 0.079830702}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_9e1e8f8435', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4ba9fc1a-4f3e-4173-84a4-0460290c8f6d-0', tool_calls=[{'name': 'browse', 'args': {'txt': 'effects of air pollution'}, 'id': '4ywwmyaaz', 'type': 'tool_call'}], usage_metadata={'input_tokens': 232, 'output_tokens': 17, 'total_tokens': 249})]

In [39]:
# Since there was a tool call, execute the tool and append the tool output in the converation

chat_history.append(tools_dict[response.tool_calls[0]["name"]].invoke(response.tool_calls[0]))
chat_history

[HumanMessage(content='Tell me about the effects of air pollution', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '4ywwmyaaz', 'function': {'arguments': '{"txt":"effects of air pollution"}', 'name': 'browse'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 232, 'total_tokens': 249, 'completion_time': 0.065984086, 'prompt_time': 0.013846616, 'queue_time': 0.19894189, 'total_time': 0.079830702}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_9e1e8f8435', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4ba9fc1a-4f3e-4173-84a4-0460290c8f6d-0', tool_calls=[{'name': 'browse', 'args': {'txt': 'effects of air pollution'}, 'id': '4ywwmyaaz', 'type': 'tool_call'}], usage_metadata={'input_tokens': 232, 'output_tokens': 17, 'total_tokens': 249}),
 ToolMessage(content='Air pollution harms health, causes acid rain, reduces sun

In [40]:
chat_history.append(HumanMessage(content="What is 10 plus 10?"))

response = llm_with_tools.invoke(chat_history)
response

AIMessage(content='20', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 300, 'total_tokens': 302, 'completion_time': 0.021310654, 'prompt_time': 0.019055902, 'queue_time': 0.191035066, 'total_time': 0.040366556}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_9e1e8f8435', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--2e650272-bd1c-4735-b101-e2a2865b5f97-0', usage_metadata={'input_tokens': 300, 'output_tokens': 2, 'total_tokens': 302})